<a href="https://colab.research.google.com/github/Marudhan/MiniProject/blob/main/HR_Mini_project_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import csv
import re
import traceback
from tabulate import tabulate
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from IPython.core.interactiveshell import InteractiveShell # Capitalize 'I' in 'IPython'
InteractiveShell.ast_node_interactivity = "all"

In [6]:
df= pd.read_csv("/content/HR_comma_sep.csv")
df.head(5)

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


How many employees in each of the department (use column "sales")

In [7]:
department_employee_count = df.groupby('sales').size().reset_index(name='num_employees')
department_employee_count.insert(0, 'Serial No', range(1, len(department_employee_count) + 1))
print("Department-wise Employee Count")
print(tabulate(department_employee_count, headers='keys', tablefmt='grid', showindex=False))


Department-wise Employee Count
+-------------+-------------+-----------------+
|   Serial No | sales       |   num_employees |
+=============+=============+=================+
|           1 | IT          |            1227 |
+-------------+-------------+-----------------+
|           2 | RandD       |             787 |
+-------------+-------------+-----------------+
|           3 | accounting  |             767 |
+-------------+-------------+-----------------+
|           4 | hr          |             739 |
+-------------+-------------+-----------------+
|           5 | management  |             630 |
+-------------+-------------+-----------------+
|           6 | marketing   |             858 |
+-------------+-------------+-----------------+
|           7 | product_mng |             902 |
+-------------+-------------+-----------------+
|           8 | sales       |            4140 |
+-------------+-------------+-----------------+
|           9 | support     |            2229 |
+--------

In [8]:
left_employees_department = df[df['left'] == 1].groupby('sales').size().reset_index(name='num_employees')
left_employees_department['rank'] = left_employees_department['num_employees'].rank(ascending=False)
left_employees_department.insert(0, 'Serial No', range(1, len(left_employees_department) + 1))
print("\nDepartment-wise Employee Count of Employees Who Left and Their Rank")
print(tabulate(left_employees_department, headers='keys', tablefmt='grid', showindex=False))


Department-wise Employee Count of Employees Who Left and Their Rank
+-------------+-------------+-----------------+--------+
|   Serial No | sales       |   num_employees |   rank |
+=============+=============+=================+========+
|           1 | IT          |             273 |      4 |
+-------------+-------------+-----------------+--------+
|           2 | RandD       |             121 |      9 |
+-------------+-------------+-----------------+--------+
|           3 | accounting  |             204 |      6 |
+-------------+-------------+-----------------+--------+
|           4 | hr          |             215 |      5 |
+-------------+-------------+-----------------+--------+
|           5 | management  |              91 |     10 |
+-------------+-------------+-----------------+--------+
|           6 | marketing   |             203 |      7 |
+-------------+-------------+-----------------+--------+
|           7 | product_mng |             198 |      8 |
+-------------+----

In [9]:

avg_time_hours = df.groupby(['sales', 'left']).agg(
    avg_time_spend_company=('time_spend_company', 'mean'),
    avg_average_monthly_hours=('average_montly_hours', 'mean')
).reset_index()
heatmap_data = avg_time_hours.pivot(index='sales', columns='left', values=['avg_time_spend_company', 'avg_average_monthly_hours'])

heatmap_data_reset = heatmap_data.reset_index()

heatmap_data_reset.insert(0, 'Serial No', range(1, len(heatmap_data_reset) + 1))

print("\nHeatmap Data for Average Time Spent and Monthly Hours by Department and Left Status")
print(tabulate(heatmap_data_reset, headers='keys', tablefmt='grid', showindex=False))



Heatmap Data for Average Time Spent and Monthly Hours by Department and Left Status
+---------------------+-----------------+---------------------------------+---------------------------------+------------------------------------+------------------------------------+
|   ('Serial No', '') | ('sales', '')   |   ('avg_time_spend_company', 0) |   ('avg_time_spend_company', 1) |   ('avg_average_monthly_hours', 0) |   ('avg_average_monthly_hours', 1) |
+=====================+=================+=================================+=================================+====================================+====================================+
|                   1 | IT              |                         3.35639 |                         3.86081 |                            198.887 |                            213.85  |
+---------------------+-----------------+---------------------------------+---------------------------------+------------------------------------+---------------------------------

In [11]:
accident_promotion_stats = df.groupby(["left", "sales"]).agg(
    count_work_accident=("Work_accident", "sum"),
    count_promotion_last_5years=("promotion_last_5years", "sum")
).reset_index()


accident_promotion_stats_sorted = accident_promotion_stats.sort_values(by="sales")

accident_promotion_stats_sorted.insert(0, 'Serial No', range(1, len(accident_promotion_stats_sorted) + 1))


print("\nCount of Work Accidents and Promotions by Department and Left Status")
print(tabulate(accident_promotion_stats_sorted, headers='keys', tablefmt='grid', showindex=False))



Count of Work Accidents and Promotions by Department and Left Status
+-------------+--------+-------------+-----------------------+-------------------------------+
|   Serial No |   left | sales       |   count_work_accident |   count_promotion_last_5years |
+=============+========+=============+=======================+===============================+
|           1 |      0 | IT          |                   155 |                             0 |
+-------------+--------+-------------+-----------------------+-------------------------------+
|           2 |      1 | IT          |                     9 |                             3 |
+-------------+--------+-------------+-----------------------+-------------------------------+
|           3 |      0 | RandD       |                   124 |                            27 |
+-------------+--------+-------------+-----------------------+-------------------------------+
|           4 |      1 | RandD       |                    10 |             

In [15]:
eval_satisfaction_stats = df.groupby(["sales", "left"]).agg(
    min_last_evaluation=("last_evaluation", "min"),
    max_last_evaluation=("last_evaluation", "max"),
    avg_last_evaluation=("last_evaluation", "mean"),
    min_satisfaction_level=("satisfaction_level", "min"),
    max_satisfaction_level=("satisfaction_level", "max"),
    avg_satisfaction_level=("satisfaction_level", "mean")
).reset_index()

eval_satisfaction_stats_sorted = eval_satisfaction_stats.sort_values(by="sales")

eval_satisfaction_stats_sorted.insert(0, 'Serial No', range(1, len(eval_satisfaction_stats_sorted) + 1))


print("\nEvaluation and Satisfaction Statistics by Department and Left Status")
print(tabulate(eval_satisfaction_stats_sorted, headers='keys', tablefmt='grid', showindex=False))


Evaluation and Satisfaction Statistics by Department and Left Status
+-------------+-------------+--------+-----------------------+-----------------------+-----------------------+--------------------------+--------------------------+--------------------------+
|   Serial No | sales       |   left |   min_last_evaluation |   max_last_evaluation |   avg_last_evaluation |   min_satisfaction_level |   max_satisfaction_level |   avg_satisfaction_level |
+=============+=============+========+=======================+=======================+=======================+==========================+==========================+==========================+
|           1 | IT          |      0 |                  0.37 |                     1 |              0.71305  |                     0.12 |                     1    |                 0.67717  |
+-------------+-------------+--------+-----------------------+-----------------------+-----------------------+--------------------------+-------------------------

In [14]:
min_max_time_hours = df.groupby(["left", "sales"]).agg(
    min_time_spend_company=("time_spend_company", "min"),
    max_time_spend_company=("time_spend_company", "max"),
    min_average_montly_hours=("average_montly_hours", "min"),
    max_average_montly_hours=("average_montly_hours", "max")
).reset_index()


min_max_time_hours_sorted = min_max_time_hours.sort_values(by="sales")

min_max_time_hours_sorted.insert(0, 'Serial No', range(1, len(min_max_time_hours_sorted) + 1))

print("\nMinimum and Maximum Time Spent and Monthly Hours by Department and Left Status")
print(tabulate(min_max_time_hours_sorted, headers='keys', tablefmt='grid', showindex=False))



Minimum and Maximum Time Spent and Monthly Hours by Department and Left Status
+-------------+--------+-------------+--------------------------+--------------------------+----------------------------+----------------------------+
|   Serial No |   left | sales       |   min_time_spend_company |   max_time_spend_company |   min_average_montly_hours |   max_average_montly_hours |
+=============+========+=============+==========================+==========================+============================+============================+
|           1 |      0 | IT          |                        2 |                       10 |                         96 |                        287 |
+-------------+--------+-------------+--------------------------+--------------------------+----------------------------+----------------------------+
|           2 |      1 | IT          |                        2 |                        6 |                        127 |                        308 |
+-------------